In [25]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('241204_01_RDD_API')
spark = SparkContext(conf = conf).getOrCreate()

In [26]:
spark

<SparkContext master=local appName=241204_01_RDD_API>

In [27]:
foods = spark.parallelize(["짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"])
foods

#병렬 처리할 수 있도록 집합으로 구성함

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [28]:
# 모두 메모리에 올림
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [31]:
#값을 기준으로 카운트 : 음식별 개수 세기
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

In [32]:
# 상위 3개만 가져오기
foods.take(3)

['짜장면', '마라탕', '짬뽕']

In [36]:
#처음 하나만 가져오기
foods.first()

'짜장면'

In [37]:
# 개수 세기
foods.count()

12

In [38]:
foods.take(foods.count())

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [58]:
# 중복을 제거한 목록
# Transformation 연산
print(foods.distinct())
print(foods.distinct().take(foods.count()))

PythonRDD[87] at RDD at PythonRDD.scala:53
['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']


In [59]:
#action 연산 : Result
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

꺼내는 행동들(collect, first)는 job에 생기는데,  
중복(distinct)와 같은 Transformation 것은 job에 생기지 않음

In [61]:
# 워커노드에서 실행하는 기능
foods.foreach(lambda x : print(x))

짜장면
마라탕
짬뽕
떡볶이
쌀국수
짬뽕
짜장면
짜장면
짜장면
라면
우동
라면


In [63]:
foods.foreach(lambda x: x)

### Narrow Operation 1 : 1 연산
filter(), map(), flatMap(), sample(), union()

In [69]:
sample_rdd = spark.parallelize([1,2,3,4,5])
sample_rdd

ParallelCollectionRDD[97] at readRDDFromFile at PythonRDD.scala:274

In [74]:
sample_rdd2 = sample_rdd.map(lambda x : x + 2)

In [75]:
sample_rdd2.collect()

[3, 4, 5, 6, 7]

In [76]:
movies = [
    '그린 북',
    '매트릭스',
    '토이 스토리',
    '캐스트 어웨이',
    '포드 V 페라리',
    '보헤미안 랩소디',
    '빽 투 더 퓨처',
    '반지의 제왕',
    '죽은 시인의 사회'
]

In [79]:
moviesRDD = spark.parallelize(movies)
moviesRDD

ParallelCollectionRDD[104] at readRDDFromFile at PythonRDD.scala:274

In [82]:
mapMovies = moviesRDD.map(lambda x : x.split(' '))

In [83]:
mapMovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [91]:
flatMapMovies = moviesRDD.flatMap(lambda x : x.split(' '))

In [92]:
flatMapMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [99]:
filterdMovies = flatMapMovies.filter(lambda x : x!= '매트릭스')
filterdMovies.collect()

['그린',
 '북',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

In [100]:
# 집합 연산
num1 = spark.parallelize([1,2,3,4,5])
num2 = spark.parallelize([4,5,6,7,8,9,10])

In [103]:
#intersection(교집합)
num1.intersection(num2).collect()

[4, 5]

In [104]:
#union(합집합)
num1.union(num2).collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [107]:
#subtract(차집합)
num1.subtract(num2).collect()


[2, 1, 3]

In [110]:
num2.subtract(num1).collect()

[6, 8, 10, 7, 9]

In [165]:
num2.subtract(num1).sortBy(lambda x : x).collect()

[6, 7, 8, 9, 10]

### sample withReplacement, fractionm, seed)  
일부를 샘플링해서 추출한다.  
withReplacement : 비복원, 복원 추출  
fraction : 기대값  
seed : 난수 추출을 위한 시드값 - 재현 가능  

## withReplacement:  
True: 데이터를 복원 추출(with replacement). 동일한 요소가 여러 번 샘플에 포함될 수 있음.  
False: 데이터를 비복원 추출(without replacement). 동일한 요소는 한 번만 샘플에 포함됨.  

## fraction:  
추출 비율을 지정.  
예: 0.7은 각 요소가 샘플링될 확률이 70%임을 의미.  

## seed (선택 사항):  
랜덤 샘플링의 시드 값. 시드를 설정하면 결과가 재현 가능함.  

numlist.sample(True, 0.5, seed=42).collect()에서 지정된 seed=42는  
Spark 작업 실행 중에만 사용되며, 메모리에 저장되거나 지속적으로 유지되지 않습니다.  

### Seed 값의 역할
#### 랜덤성 제어:
seed는 샘플링 시 랜덤 번호 생성기의 초기값을 설정합니다.  
이를 통해 샘플링 결과가 재현 가능해집니다.  
동일한 seed를 사용하면 같은 입력 데이터에서 항상 동일한 샘플링 결과를 얻을 수 있습니다.   
#### 메모리 관리:  
seed는 Spark의 샘플링 작업이 실행될 때만 사용됩니다. 작업이 끝나면 메모리에서 해제됩니다.  
따라서, seed=42가 지정되어 있다고 해서 Spark 컨텍스트나 메모리에 그 값이 계속 저장되지는 않습니다.  


In [134]:
numlist = num1.union(num2)
numlist.collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [149]:
#전체 개수 중에, 50%의 확률만큼의 개수만큼만 꺼냄
numlist.sample(True, 0.5).collect()

[2, 4, 7, 9, 10]

In [157]:
#전체 개수 중에, 30%의 확률만큼의 개수만큼만 꺼냄
numlist.sample(True, 0.3).collect()

[4, 4, 6]

In [162]:
#전체 개수 중에, 70%의 확률만큼의 개수만큼만 꺼냄
#True는 복원 추출이라, 중복된 값이 있을 수 있음 ex([5,5])
numlist.sample(True, 0.7).collect()

[1, 3, 5, 4, 6, 6, 6, 7, 7]

In [160]:
#보통 0.4를 많이 줌
# seed가 있으니 재현이 가능한 샘플
# False : 동일 요소 한번만 샘플링
numlist.sample(True, 0.5, seed=42).collect()

[5, 6, 6]

### wide transformation
- groupby
- reduce

In [166]:
foods.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

In [177]:
foodsGroup = foods.groupBy(lambda x : x[0])
res = foodsGroup.collect()

#이러면 (key, value) 형태로 나옴

In [186]:
for k , v in res:
    print(k, list(v))    

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']


In [93]:
movies_100 = [
"The Shawshank Redemption", "The Godfather", "The Dark Knight", "12 Angry Men",
"Schindler's List", "The Lord of the Rings: The Return of the King", "Pulp Fiction",
"The Good, the Bad and the Ugly", "Fight Club", "Forrest Gump", "Inception",
"Star Wars: Episode V - The Empire Strikes Back", "The Matrix", "Goodfellas",
"One Flew Over the Cuckoo's Nest", "Se7en", "It's a Wonderful Life", "Seven Samurai",
"The Silence of the Lambs", "City of God", "Saving Private Ryan", "Interstellar",
"The Green Mile", "Spirited Away", "Parasite", "Léon: The Professional",
"The Usual Suspects", "Harakiri", "The Lion King", "Back to the Future",
"The Pianist", "Terminator 2: Judgment Day", "American History X", "Modern Times",
"Psycho", "Gladiator", "City Lights", "The Departed", "The Intouchables",
"Whiplash", "The Prestige", "Casablanca", "Once Upon a Time in the West",
"Rear Window", "Cinema Paradiso", "Alien", "Apocalypse Now", "Memento",
"Indiana Jones and the Raiders of the Lost Ark", "The Great Dictator",
"The Lives of Others", "Django Unchained", "Paths of Glory", "The Shining",
"WALL·E", "Sunset Blvd.", "Witness for the Prosecution", "Oldboy", "Princess Mononoke",
"Dr. Strangelove", "The Dark Knight Rises", "Once Upon a Time in America",
"Aliens", "Coco", "Avengers: Infinity War", "American Beauty", "Braveheart",
"Das Boot", "3 Idiots", "Toy Story", "Amadeus", "Inglourious Basterds",
"Good Will Hunting", "Star Wars: Episode IV - A New Hope", "Reservoir Dogs",
"2001: A Space Odyssey", "Vertigo", "M", "Citizen Kane", "Requiem for a Dream",
"The Hunt", "Eternal Sunshine of the Spotless Mind", "Full Metal Jacket",
"A Clockwork Orange", "Taxi Driver", "Lawrence of Arabia", "Double Indemnity",
"Amélie", "Like Stars on Earth", "Singin' in the Rain", "Bicycle Thieves",
"Snatch", "Scarface", "3 Idiots", "Toy Story 3", "The Sting", "Metropolis",
"For a Few Dollars More", "L.A. Confidential", "Rashomon", "Monty Python and the Holy Grail"
]

In [29]:
#stop은 항상 나중에 챙기기
#spark.stop()

In [187]:
spark.stop()